In [2]:
import numpy as np 
import networkx as nx
import matplotlib.pyplot as plt
import random as rand
from collections import Counter

In [37]:
G = nx.barabasi_albert_graph(n=1000,m=2)

k_centrality = nx.degree_centrality(G)
k_centrality = np.fromiter(k_centrality.values(), float)

In [5]:
list(Counter(nx.get_node_attributes(G, "status").values()).values())

[]

In [36]:
def maki_thompson_rumour_model(graph: nx.Graph, gamma: float, alpha: float, time_limit: int, number_infected=1, all_time_values=False, starting_nodes=[]):

    #Clear status of the nodes of the given graph
    #"status" tells the type of node (I - Ignorant, S - Spreader, R - Stifler)
    #"visit" tells if the node should be visited in a cycle or not

    node_status={key: "I" for key in graph.nodes}
    node_visit={key: False for key in graph.nodes}
    nx.set_node_attributes(graph, node_status, "status")
    nx.set_node_attributes(graph, node_visit, "visit")

    #Obtain a random sample of size "number_infected" out of the nodes of the given graph
    if not(starting_nodes):
        starting_nodes = rand.sample(list(graph.nodes), number_infected)

    for node in starting_nodes:
        graph.nodes[node]["status"]="S"
        graph.nodes[node]["visit"]=True

        #All neighbours of spreaders should be visited in the cycle for possible changes
        for neighbor in graph.neighbors(node):
            graph.nodes[neighbor]["visit"]=True

    t=0

    #Counter for every status type, for each time value "t"
    counter=Counter(nx.get_node_attributes(graph, "status").values())
    status_count=[{"I": counter["I"], "S": counter["S"], "R": counter["R"]}]

    #Checks if there is still nodes worth visiting in the next iteration
    continue_visitingQ=True
    while t < time_limit and continue_visitingQ:
        #Only apply node_changes at the end of the cycle, the changes have to be done all at the same time.
        node_changes=[]
        for node in graph.nodes(data=True):
            if node[1]["visit"]==True:
                #Counting the amount of each type of nodes in the neighbourhood of the visited node
                neighbours_status_count={"I":0, "S":0, "R":0}
                for neighbor in graph.neighbors(node[0]):
                    neighbours_status_count[graph.nodes[neighbor]["status"]]+=1

                #If this value is less than the probability of transforming then the status of the node changes
                transform_value = rand.uniform(0,1)

                #Transformation of the node status
                if node[1]["status"]=="I":
                    #(1-gamma)**neighbours_status_count["S"] gives the probability of the event that no spreader transmits the rumour. We only need one of the spreaders to pass the rumour to transform the ignorant into spreader
                    transform_threshold = 1-(1-gamma)**neighbours_status_count["S"]
                    if transform_value <= transform_threshold:
                        node_changes.append((node[0], "S", True))
                        for neighbor in graph.neighbors(node[0]):
                            if graph.nodes[neighbor]["status"]!="R":
                                node_changes.append((neighbor, graph.nodes[neighbor]["status"], True))
                elif node[1]["status"]=="S":
                     #Whenever we are transforming a spreader into a stiffler the stifflers and the spreaders contribute the same to the probability of transforming the spreader
                    transform_threshold = 1-(1-alpha)**(neighbours_status_count["S"] + neighbours_status_count["R"])
                    if transform_value <= transform_threshold:
                        node_changes.append((node[0], "R", False))
        
        #Applying all changes to the graph
        for node_change in node_changes:
            graph.nodes[node_change[0]]["status"]=node_change[1]
            graph.nodes[node_change[0]]["visit"]=node_change[2]
        
        counter=Counter(nx.get_node_attributes(graph, "status").values())
        status_count.append({"I": counter["I"], "S": counter["S"], "R": counter["R"]})

        if not(all_time_values) and counter["S"]==0:
            continue_visitingQ=False
        t+=1
    return status_count

In [38]:
maki_thompson_rumour_model(G, gamma=0.1, alpha=0.01, time_limit = 1000, number_infected=2, all_time_values=False)

[{'I': 998, 'S': 2, 'R': 0},
 {'I': 998, 'S': 2, 'R': 0},
 {'I': 997, 'S': 3, 'R': 0},
 {'I': 997, 'S': 3, 'R': 0},
 {'I': 997, 'S': 3, 'R': 0},
 {'I': 997, 'S': 3, 'R': 0},
 {'I': 996, 'S': 4, 'R': 0},
 {'I': 994, 'S': 6, 'R': 0},
 {'I': 991, 'S': 9, 'R': 0},
 {'I': 989, 'S': 11, 'R': 0},
 {'I': 986, 'S': 14, 'R': 0},
 {'I': 975, 'S': 25, 'R': 0},
 {'I': 950, 'S': 50, 'R': 0},
 {'I': 922, 'S': 78, 'R': 0},
 {'I': 895, 'S': 105, 'R': 0},
 {'I': 866, 'S': 134, 'R': 0},
 {'I': 828, 'S': 167, 'R': 5},
 {'I': 787, 'S': 207, 'R': 6},
 {'I': 743, 'S': 247, 'R': 10},
 {'I': 691, 'S': 296, 'R': 13},
 {'I': 630, 'S': 352, 'R': 18},
 {'I': 578, 'S': 402, 'R': 20},
 {'I': 536, 'S': 432, 'R': 32},
 {'I': 484, 'S': 467, 'R': 49},
 {'I': 437, 'S': 502, 'R': 61},
 {'I': 375, 'S': 552, 'R': 73},
 {'I': 326, 'S': 588, 'R': 86},
 {'I': 289, 'S': 607, 'R': 104},
 {'I': 253, 'S': 627, 'R': 120},
 {'I': 217, 'S': 643, 'R': 140},
 {'I': 191, 'S': 657, 'R': 152},
 {'I': 167, 'S': 661, 'R': 172},
 {'I': 148, 